In [0]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import scipy.sparse as sp
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import random

In [0]:
def load_restaurant_dataset():
    path = 'dataset_ubicomp2013_checkins.txt'
#     lines = (line.decode('utf-8') for line in path)
    infile = open(path, 'r')
    a = set()
    b = set()
    edges = []
    for line in infile:
        s=line.strip().split(None)
        u=-1*int(s.pop(0)) -10
        v=int(s.pop(0))
        a.add(u)
        b.add(v)
        edges.append((u,v))
    top_nodes = {}
    bottom_nodes = {}
    count = 0 
    for x in a:
        top_nodes[x] = count
        count = count + 1
    count  = 0    
    for y in b:
        bottom_nodes[y] = count
        count  = count + 1
    
    A = np.zeros((len(a),len(b)))
    for edge in edges:
        e1 = top_nodes[edge[0]]
        e2 = bottom_nodes[edge[1]]
        A[e1, e2] = 1
    
    A = np.dot(A,A.T)
#     print(A[:35,:35])
    for i in range(0,A.shape[0]):  #making numpy matrix undirected graph type
        for j in range(0,A.shape[1]):
            if i == j :
                A[i,j] = 0
            else:
                if A[i,j] > 0:
                    A[i,j] = 1
                    
    G=nx.from_numpy_matrix(A)
    return G

In [0]:
graph = load_restaurant_dataset()

In [0]:
nx.info(graph)

'Name: \nType: Graph\nNumber of nodes: 2060\nNumber of edges: 58810\nAverage degree:  57.0971'

In [0]:
from similarities import *
from preprocessing import mask_test_edges

In [0]:
np.random.seed(0)
adj_sparse = nx.to_scipy_sparse_matrix(graph)

In [0]:
edges = list(nx.edges(graph))

In [0]:
len(edges)

58810

In [0]:
non_edges = list(nx.non_edges(graph))

In [0]:
len(non_edges)

2061960

In [0]:
selected_new_edges = random.sample(non_edges,58810)

In [0]:
len(selected_new_edges)

58810

In [0]:
g_train = nx.Graph()
g_train.add_edges_from(graph.edges)
g_train.add_edges_from(selected_new_edges)
nx.info(g_train)

'Name: \nType: Graph\nNumber of nodes: 2060\nNumber of edges: 117620\nAverage degree: 114.1942'

In [0]:
nodes = list(g_train.nodes())

In [0]:
len(nodes)

2060

In [0]:
labels_all = [1]*58810 + [0]*58810

In [0]:
actual_edge_scores = []
new_edge_scores = []
for ed in edges:
  actual_edge_scores.append(PA_score(g_train,ed[0],ed[1]))
for ed in selected_new_edges:
  new_edge_scores.append(PA_score(g_train,ed[0],ed[1]))
  
preds_all = actual_edge_scores + new_edge_scores  
roc_score = roc_auc_score(labels_all, preds_all)
ap_score = average_precision_score(labels_all, preds_all)
  

In [0]:
print("Preferential attachment model")
print('Test ROC score: ', str(roc_score))
print('Test AP score: ', str(ap_score))

Preferential attachment model
Test ROC score:  0.8280088426119336
Test AP score:  0.8240212735188608


In [0]:
actual_edge_scores = []
new_edge_scores = []
for ed in edges:
  actual_edge_scores.append(AA_score(g_train,ed[0],ed[1]))
for ed in selected_new_edges:
  new_edge_scores.append(AA_score(g_train,ed[0],ed[1]))
  
preds_all = actual_edge_scores + new_edge_scores  
roc_score = roc_auc_score(labels_all, preds_all)
ap_score = average_precision_score(labels_all, preds_all)

In [0]:
print("Adamic adar model")
print('Test ROC score: ', str(roc_score))
print('Test AP score: ', str(ap_score))

Adamic adar model
Test ROC score:  0.9472728300200767
Test AP score:  0.9486010195908283


In [0]:
actual_edge_scores = []
new_edge_scores = []
for ed in edges:
  actual_edge_scores.append(CN_score(g_train,ed[0],ed[1]))
for ed in selected_new_edges:
  new_edge_scores.append(CN_score(g_train,ed[0],ed[1]))
  
preds_all = actual_edge_scores + new_edge_scores  
roc_score = roc_auc_score(labels_all, preds_all)
ap_score = average_precision_score(labels_all, preds_all)

In [0]:
print("Common Neighbors model")
print('Test ROC score: ', str(roc_score))
print('Test AP score: ', str(ap_score))

Common Neighbors model
Test ROC score:  0.9448759703339148
Test AP score:  0.943188841927863


In [0]:
actual_edge_scores = []
new_edge_scores = []
for ed in edges:
  actual_edge_scores.append(RA_score(g_train,ed[0],ed[1]))
for ed in selected_new_edges:
  new_edge_scores.append(RA_score(g_train,ed[0],ed[1]))
  
preds_all = actual_edge_scores + new_edge_scores  
roc_score = roc_auc_score(labels_all, preds_all)
ap_score = average_precision_score(labels_all, preds_all)

In [0]:
print("Resource Allocation model")
print('Test ROC score: ', str(roc_score))
print('Test AP score: ', str(ap_score))

Resource Allocation model
Test ROC score:  0.9525555260382903
Test AP score:  0.9548965675307882


In [0]:
actual_edge_scores = []
new_edge_scores = []
for ed in edges:
  actual_edge_scores.append(JC_score(g_train,ed[0],ed[1]))
for ed in selected_new_edges:
  new_edge_scores.append(JC_score(g_train,ed[0],ed[1]))
  
preds_all = actual_edge_scores + new_edge_scores  
roc_score = roc_auc_score(labels_all, preds_all)
ap_score = average_precision_score(labels_all, preds_all)

In [0]:
print("Jaccard Coef model")
print('Test ROC score: ', str(roc_score))
print('Test AP score: ', str(ap_score))

Jaccard Coef model
Test ROC score:  0.9612010597244371
Test AP score:  0.9660288187732504


In [0]:
print(list(g_train.nodes())[:10])

[0, 21, 83, 89, 110, 125, 170, 171, 176, 181]


In [0]:
print(nodes[:10])

[0, 21, 83, 89, 110, 125, 170, 171, 176, 181]


In [0]:
rpr_mat = rpr_matrix(g_train)

In [0]:
def rp_score(u,v):
# #   print(u,v)
#   u = nodes.index(u)
#   v = nodes.index(v)
#   print(u,v)
#   return min(rpr_mat[u,v],rpr_mat[v,u])
  return min(rp_dict[(u,v)] , rp_dict[(v,u)])

In [0]:
rp_dict = rpr_dict(g_train)

In [0]:
actual_edge_scores = []
new_edge_scores = []
for ed in edges:
  actual_edge_scores.append(rp_score(ed[0],ed[1]))
for ed in selected_new_edges:
  new_edge_scores.append(rp_score(ed[0],ed[1]))
  
preds_all = actual_edge_scores + new_edge_scores  
roc_score = roc_auc_score(labels_all, preds_all)
ap_score = average_precision_score(labels_all, preds_all)

In [0]:
print(actual_edge_scores[:5])
print(new_edge_scores[:5])

[0.0008071896094874578, 0.0014050735258175043, 0.0011478110168308313, 0.0009537025168378985, 0.0013288288786438185]
[0.0013411172809520697, 0.001088333938241924, 0.0013407539418909974, 0.0010487556364400716, 0.0016769431924553992]


In [0]:
print("Rooted Pagerank model")
print('Test ROC score: ', str(roc_score))
print('Test AP score: ', str(ap_score))

Rooted Pagerank model
Test ROC score:  0.3835188733435897
Test AP score:  0.40488883838602463


In [0]:
def katz_score(graph,beta=0.004):
    # non_edges = nx.non_edges(graph)
    A = nx.to_numpy_matrix(graph)
    # print(A)
    w, v = np.linalg.eigh(A)
    lambda1 = max([abs(x) for x in w])   # beta should be less than 1/lambda1
    # print(1/lambda1)
    if beta >= 1/lambda1 :
        raise ValueError('beta should be less than 1/lambda, lambda being the eigenvalue with largest magnitude')
    I = np.eye(A.shape[0])
    S = np.linalg.inv(I - beta * A) - I
    return S

In [0]:
katz_values = katz_score(g_train)

In [0]:
def Katz_score(u,v):
  u = nodes.index(u)
  v = nodes.index(v)
  return katz_values[u,v] + katz_values[v,u]

In [0]:
actual_edge_scores = []
new_edge_scores = []
for ed in edges:
  actual_edge_scores.append(Katz_score(ed[0],ed[1]))
for ed in selected_new_edges:
  new_edge_scores.append(Katz_score(ed[0],ed[1]))
  
preds_all = actual_edge_scores + new_edge_scores  
roc_score = roc_auc_score(labels_all, preds_all)
ap_score = average_precision_score(labels_all, preds_all)

In [0]:
print("Katz model")
print('Test ROC score: ', str(roc_score))
print('Test AP score: ', str(ap_score))

Katz model
Test ROC score:  0.9059285388742625
Test AP score:  0.9038693091862358
